# Задание по курсу «Дискретная оптимизация», МФТИ, весна 2017

## Задача 3-1. Задача TSP: инкрементальные алгоритмы.

В этой задаче Вам предлагается сравнить алгоритмы Nearest Neighbour и Nearest Insertion в задаче Euclidean TSP.

**Даны:**
* Координаты точек плоскости, являющихся вершинами графа.

**Найти:**
* Перестановку вершин, задающих минимальный по длине гамильтонов цикл в графе.

Сделайте следующее:
* Скачайте файл [`tsp-instances.zip`](https://github.com/dainiak/discrete-optimization-course/raw/master/tsp-instances.zip) и разархивируйте из него файлы со входами задачи TSP.
* Реализуйте функции `solve_tsp_nearest_neighbour` и `solve_tsp_nearest_insertion`.
* Запустите функцию `run_all()`, чтобы протестировать свой код и сравнить качество решений, получаемых Nearest Neighbour и Nearest Insertion. Сильно ли они отличаются? Запишите свои качественные выводы в 1-2 предложениях в последней ячейке ipynb-файла.

In [1]:
def read_tsp_instance(filename):
    with open(filename, 'r') as file:
        coordinates = []
        for line in file:
            line = line.strip().lower()
            if line.startswith('dimension'):
                coordinates = [(0,0)] * int(line.split()[-1])
            tokens = line.split()
            if len(tokens) == 3 and tokens[0].isdecimal():
                tokens = line.split()
                coordinates[int(tokens[0])-1] = tuple(map(float, tokens[1:]))
        return coordinates

from math import sqrt

def euclidean_distance(point1, point2):
    return sqrt((point1[0]-point2[0]) ** 2 + (point1[1]-point2[1]) ** 2)
    
def calculate_tour_length(instance, permutation):
    n = len(permutation)
    return sum(euclidean_distance(instance[permutation[i]], instance[permutation[(i+1) % n]]) for i in range(len(permutation)))

In [118]:
def solve_tsp_nearest_neighbour(instance):
    n = len(instance)
    cycle = [0]
    used = [False] * n
    used[0] = True
    for i in range(n - 1):
        min_dist = 10 ** 18
        nearest = 0
        for j in range(n):
            if not used[j]:
                d = euclidean_distance(instance[cycle[i]], instance[j])
                if d < min_dist:
                    min_dist = d
                    nearest = j
        cycle.append(nearest)
        used[nearest] = True
    return cycle

for file in ['d198.tsp', 'd493.tsp', 'd657.tsp', 'd2103.tsp', 'pr107.tsp', 'pr152.tsp', 'pr439.tsp']:
    instance = read_tsp_instance(file)
    solution = solve_tsp_nearest_neighbour(instance)
    print(calculate_tour_length(instance, solution), calculate_tour_length(instance, range(len(instance))))

18620.073812019415 22514.12084286643


In [13]:
import numpy as np

In [117]:
def solve_tsp_nearest_insertion(instance):
    def w(i, j):
        return euclidean_distance(instance[i], instance[j])
    
    n = len(instance)
    cycle = [0]
    used = [False] * n
    used[0] = True
    dst = [w(0, i) for i in range(1, n)]
    cycle.append(dst.index(min(dst)) + 1)
    used[cycle[1]] = True
    a = list(range(1, n))
    not_in_cycle = a[:cycle[1] - 1] + a[cycle[1]:]
    for i in range(n - 2):
        k = len(cycle)
        m = n - len(cycle)
        #Использую numpy.arrays, чтобы ускорить вычисления - код работает 2 минуты вместо 2 часов
        A = np.array([list(instance[i]) for i in cycle]).T
        B = np.array([list(instance[i]) for i in not_in_cycle]).T
        A1 = np.tile(A, m).reshape((2, m, k))
        A1 = np.array([x.T for x in A1])
        B1 = np.tile(B, k).reshape((2, k, m))
        D = np.sqrt((A1[0] - B1[0]) ** 2 + (A1[1] - B1[1]) ** 2)# Массив попарных расстояний между всеми вершинами, \
            #включенными в цикл и всеми не включенными
        dist_v_u1_v_u2 = D + np.roll(D, 1, axis=0)#dist_v_u1_v_u2[i][j] = D[i][j] - D[i - 1][j]
        neigh_dist_in_cycle = np.array([w(cycle[i - 1], cycle[i]) for i in range(k)])
        M = dist_v_u1_v_u2.T - neigh_dist_in_cycle#w(v, u1) + w(v, u2) - w(u1, u2)
        p = np.argmin(dist_v_u1_v_u2.T - neigh_dist_in_cycle)
        min_delta = np.min(dist_v_u1_v_u2.T - neigh_dist_in_cycle)
        v = not_in_cycle[p // k]
        u1 = cycle[p % k - 1]
        u2 = cycle[p % k]
        if i == 0:
            for j in range(k):
                for t in range(m):
                    cur_d = w(cycle[j], not_in_cycle[t]) + w(cycle[j - 1], not_in_cycle[t]) - w(cycle[j - 1], cycle[j])
                    assert(cur_d > min_delta - 1e-8)
        assert(abs(w(v, u1) + w(v, u2) - w(u1, u2) - min_delta) < 1e-8)
        cycle = cycle[:p % k] + [v] + cycle[p % k:]
        not_in_cycle = not_in_cycle[:p // k] + not_in_cycle[p // k + 1:]
    assert(len(cycle) == n)
    return cycle

In [60]:
a = np.array([[1, 2, 3, 4, 5], [6, 7, 8, 9, 10], [11, 12, 13, 14, 15]])
print(np.roll(a, 1, axis=0))

[[11 12 13 14 15]
 [ 1  2  3  4  5]
 [ 6  7  8  9 10]]


In [102]:
import time
from os.path import exists

def run_all():
    instance_filenames = ['d198.tsp', 'd493.tsp', 'd657.tsp', 'd2103.tsp', 'pr107.tsp', 'pr152.tsp', 'pr439.tsp']
    for filename in instance_filenames:
        if not exists(filename):
            print('File not found: “{}”. Skipping this instance.'.format(filename))
            continue
        instance = read_tsp_instance(filename)
        print('Solving instance {}…'.format(filename), end='')
        time_start = time.monotonic()
        quality_nn = calculate_tour_length(instance, solve_tsp_nearest_neighbour(instance))
        time_nn = time.monotonic()-time_start
        time_start = time.monotonic()
        quality_ni = calculate_tour_length(instance, solve_tsp_nearest_insertion(instance))
        time_ni = time.monotonic()-time_start
        print(' done in {:.2} seconds with tour length {} using NN and in {:.2} seconds with tour length {} using NI'.format(time_nn, int(quality_nn), time_ni, int(quality_ni)))

In [103]:
run_all()

Solving instance d198.tsp… done in 0.029 seconds with tour length 18620 using NN and in 0.16 seconds with tour length 17631 using NI
Solving instance d493.tsp… done in 0.18 seconds with tour length 43646 using NN and in 1.5 seconds with tour length 39982 using NI
Solving instance d657.tsp… done in 0.32 seconds with tour length 62176 using NN and in 3.9 seconds with tour length 57906 using NI
Solving instance d2103.tsp… done in 3.2 seconds with tour length 87468 using NN and in 1.4e+02 seconds with tour length 87530 using NI
Solving instance pr107.tsp… done in 0.0085 seconds with tour length 46678 using NN and in 0.05 seconds with tour length 51667 using NI
Solving instance pr152.tsp… done in 0.017 seconds with tour length 85702 using NN and in 0.093 seconds with tour length 88530 using NI
Solving instance pr439.tsp… done in 0.14 seconds with tour length 131282 using NN and in 1.1 seconds with tour length 130067 using NI


## Выводы

В целом Nearest Insertion дает результат чуть лучше, но не всегда. Но и работает NI дольше - за $O(n^{3})$ в наивной реализации, в то время как NN - за $O(n^{2})$ Сравнивая результаты с тривиальной перестановкой, убедился, что алгоритмы неплохо оптимизируют цикл.